In [12]:
import os
import sys
import pandas as pd
import logging
import openrouteservice
from shapely.geometry import Polygon, Point, MultiLineString, MultiPolygon
import time
import geopandas as gpd


logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))
sys.path.append("../scripts")
from distance_proximity import proximity, travel_time

api_key = os.environ["ORS_KEY"]


# CALL_RANGE = range(0 + 200 * ITER, 200 + 200 * ITER, 1)
CALL_RANGE = range(12000, 15272, 1)


school_df = pd.read_csv("../data/raw/school.csv", encoding="cp1252")


## School

In [17]:
# # Once off to create column for travel time to schoo
# scraped_df = pd.read_json(
#     "../data/raw/scrape_2022-09-02_04-12-25-567025.json", orient="records"
# )
# scraped_df['school_duration'] = 0
# scraped_df['school_distance'] = 0
# scraped_df.to_csv("../data/curated/scraped_with_school_tt.csv")


In [10]:
scraped_school_df = pd.read_csv("../data/curated/scraped_with_school_tt.csv")


FileNotFoundError: [Errno 2] No such file or directory: '../data/curated/scraped_with_school_tt.csv'

In [48]:
start_time = time.time()
for i in CALL_RANGE:
    try:
        # -37.7751338, 144.7970672
        candidate_1, candidate_2 = proximity(
            scraped_school_df.iloc[i, -4],
            scraped_school_df.iloc[i, -3],
            school_df.Y.values.tolist(),
            school_df.X.values.tolist(),
        )

        # 144.7970672, -37.7751338,
        dis_1, dur_1 = travel_time(
            scraped_school_df.iloc[i, -3],
            scraped_school_df.iloc[i, -4],
            school_df.iloc[candidate_1]["X"],
            school_df.iloc[candidate_1]["Y"],
            api_key,
        )
        dis_2, dur_2 = travel_time(
            scraped_school_df.iloc[i, -3],
            scraped_school_df.iloc[i, -4],
            school_df.iloc[candidate_2]["X"],
            school_df.iloc[candidate_2]["Y"],
            api_key,
        )
        if dur_1 > dur_2:
            scraped_school_df.at[i, "school_duration"] = dur_2 / 60
            scraped_school_df.at[i, "school_distance"] = dis_2
        else:
            scraped_school_df.at[i, "school_duration"] = dur_1 / 60
            scraped_school_df.at[i, "school_distance"] = dis_1
        logging.info(f"Finished property index {i}")
    except Exception as e:
        logging.error(e)
        continue

end_time = time.time()
logging.info(f"Time taken, {(end_time - start_time)/60} mins")

scraped_school_df.to_csv("../data/curated/scraped_with_school_tt.csv")


INFO:root:Finished property index 12000
INFO:root:Finished property index 12001
INFO:root:Finished property index 12002
INFO:root:Finished property index 12003
INFO:root:Finished property index 12004
INFO:root:Finished property index 12005
INFO:root:Finished property index 12006
INFO:root:Finished property index 12007
INFO:root:Finished property index 12008
INFO:root:Finished property index 12009
/Users/matthewrush/virtualenv/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/matthewrush/virtualenv/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/matthewrush/virtualenv/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Ra

## Parks

In [13]:
from pyrosm import OSM, get_data
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

# Get test data
fp = get_data("melbourne")

osm = OSM(fp)

# Read POIs with custom filter B
park_filter = {
    "leisure": [
        "park",
        "dog_park",
        "garden",
        "nature_reserve",
        "park;playground",
        "recreation_ground",
        "reserve",
    ]
}

shop_filter = {
    "shop": [
        "department_store",
        "mall",
        "garden",
        "nature_reserve",
        "park;playground",
        "recreation_ground",
        "reserve",
    ]
}
park_pois = osm.get_pois(custom_filter=park_filter)

df = pd.read_csv("../data/curated/property_to_school.csv")
scraped_df = pd.read_json("../data/raw/scrape_2022-09-02_04-12-25-567025.json")
CALL_RANGE = range(9500, 15271)


/Users/matthewrush/virtualenv/lib/python3.10/site-packages/pyrosm/pyrosm.py:576: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gdf = get_poi_data(


In [3]:
df["park_duration"] = 0
df["park_distance"] = 0


In [7]:
geom = park_pois["geometry"].values.tolist()

park_lats = []
park_longs = []

for i in geom:
    if type(i) == Point:
        x, y = i.xy
        x = x[0]
        y = y[0]
    elif type(i) == Polygon:
        x, y = i.exterior.coords.xy
        x = x[0]
        y = y[0]
    elif type(i) == MultiPolygon:
        x, y = i.geoms[0].exterior.coords.xy
        x = x[0]
        y = y[0]
    park_lats.append(y)
    park_longs.append(x)


In [14]:
start_time = time.time()
for i in CALL_RANGE:
    try:
        # -37.7751338, 144.7970672
        candidate = proximity(
            scraped_df.iloc[i, -2],
            scraped_df.iloc[i, -1],
            park_lats,
            park_longs,
            top_only=True,
        )
        dis, dur = travel_time(
            scraped_df.iloc[i, -1],
            scraped_df.iloc[i, -2],
            park_longs[candidate],
            park_lats[candidate],
            api_key,
        )

        df.at[i, "park_duration"] = dur / 60
        df.at[i, "park_distance"] = dis
        logging.info(f"Finished property index {i}")
    except Exception as e:
        logging.error(e)
        continue

end_time = time.time()
logging.info(f"Time taken, {(end_time - start_time)/60} mins")

df.to_csv("../data/curated/property_to_school.csv")


INFO:root:Finished property index 9500
ERROR:root:404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 1: 145.0293500 -37.9979169.'}, 'info': {'engine': {'version': '6.7.0', 'build_date': '2022-02-18T19:37:41Z'}, 'timestamp': 1663161656000}})
INFO:root:Finished property index 9502
INFO:root:Finished property index 9503
INFO:root:Finished property index 9504
INFO:root:Finished property index 9505
INFO:root:Finished property index 9506
ERROR:root:404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 1: 144.6807019 -37.9795613.'}, 'info': {'engine': {'version': '6.7.0', 'build_date': '2022-02-18T19:37:41Z'}, 'timestamp': 1663161672458}})
INFO:root:Finished property index 9508
INFO:root:Finished property index 9509
INFO:root:Finished property index 9510
INFO:root:Finished property index 9511
ERROR:root:404 ({'error': {'code': 2010, 'message': 

# PTV

In [ ]:
regbusstop_df = gpd.read_file("../data/raw/PUBLIC_TRANSPORT/PTV_REGIONAL_BUS_STOP.shp")
regbusstop_df["TICKETZONE"] = 0
bus_stopdf = gpd.read_file("../data/raw/PUBLIC_TRANSPORT/PTV_METRO_BUS_STOP.shp")

tramstop_df = gpd.read_file("../data/raw/PUBLIC_TRANSPORT/PTV_METRO_TRAM_STOP.shp")

trainst_df = gpd.read_file("../data/raw/PUBLIC_TRANSPORT/PTV_METRO_TRAIN_STATION.shp")
regtrainst_df = gpd.read_file(
    "../data/raw/PUBLIC_TRANSPORT/PTV_REGIONAL_TRAIN_STATION.shp"
)
regtrainst_df["TICKETZONE"] = 0
regtrainst_df["ROUTEUSSP"] = 0

fulll_bustop = pd.concat([bus_stopdf, regbusstop_df])
full_ptvstop = pd.concat([fulll_bustop, tramstop_df])
fulll_ptvstation = pd.concat([trainst_df, regtrainst_df])


In [ ]:
df = pd.read_json("../data/raw/scrape_2022-09-02_04-12-25-567025.json")

api_key = os.environ["ORS_KEY"]

# CHANGE THIS EACH CYCLE 7638 now
CALL_RANGE = range(7638, 15272)


In [ ]:
##ONLY RUN THIS THE FIRST TIME. OUTPUT WILL BE OVERWRITTEN OTHERWISE. YOU HAVE BEEN WARNED

# url_col = df['url']
df.drop(df.columns.difference(["url", "latitude", "longitude"]), 1, inplace=True)
df["train_duration"] = 0
df["train_distance"] = 0
df
df.to_csv("../data/curated/ptv_to_prop.csv", index=False)


In [ ]:
# run once
property_to_ptv = pd.read_csv("../data/curated/ptv_to_prop.csv")
# print(property_to_ptv.head(5))


### Distance to train stations

In [ ]:
train_lats = fulll_ptvstation["LATITUDE"].values.tolist()
train_longs = fulll_ptvstation["LONGITUDE"].values.tolist()


In [ ]:
start_time = time.time()

for i in CALL_RANGE:
    try:
        # -37.7751338, 144.7970672
        # This returns the index of the closest stops (no api call, coordinates maths)
        candidate = proximity(
            property_to_ptv.iloc[i, -4],
            property_to_ptv.iloc[i, -3],
            train_lats,
            train_longs,
            top_only=True,
        )
        # This makes the call
        # 144.7970672, -37.7751338,
        dis, dur = travel_time(
            property_to_ptv.iloc[i, -3],
            property_to_ptv.iloc[i, -4],
            train_longs[candidate],
            train_lats[candidate],
            api_key,
        )

        property_to_ptv.at[i, "train_duration"] = dur / 60
        property_to_ptv.at[i, "train_distance"] = dis
        logging.info(f"Finished property index {i}")
    except Exception as e:
        logging.error(e)
        continue

end_time = time.time()
logging.info(f"Time taken, {(end_time - start_time)/60} mins")

property_to_ptv.to_csv("../data/curated/ptv_to_prop.csv")


### Distance to bus and tram

In [ ]:
stops_lats = full_ptvstop["LATITUDE"].values.tolist()
stop_longs = full_ptvstop["LONGITUDE"].values.tolist()


In [ ]:
CALL_RANGE2 = range(11200, 15272)


In [ ]:
start_time = time.time()

for i in CALL_RANGE2:
    try:
        # -37.7751338, 144.7970672
        # This returns the index of the closest stops (no api call, coordinates maths)
        candidate = proximity(
            property_to_ptv.iloc[i, -6],
            property_to_ptv.iloc[i, -5],
            stops_lats,
            stop_longs,
            top_only=True,
        )
        # This makes the call
        # 144.7970672, -37.7751338,
        dis, dur = travel_time(
            property_to_ptv.iloc[i, -5],
            property_to_ptv.iloc[i, -6],
            stop_longs[candidate],
            stops_lats[candidate],
            api_key,
        )

        property_to_ptv.at[i, "stop_duration"] = dur / 60
        property_to_ptv.at[i, "stop_distance"] = dis
        logging.info(f"Finished property index {i}")
    except Exception as e:
        logging.error(e)
        continue

end_time = time.time()
logging.info(f"Time taken, {(end_time - start_time)/60} mins")

property_to_ptv.to_csv("../data/curated/ptv_to_prop.csv", index=False)
